# Tumor Image classification

## Data Preparation

Data Augmentation is performed on the given dataset. ImageDataGenerator function in Keras can used for preprocessing the data.

The images in the given dataset didnt require any augmentation as the dataset had object of interest in focus. The data was imbalanced even after combining classes 0 , 1 into class 0 and classes 2 , 3 into class 1. Classes 1 was about half the size of class 0. Hence Class 1 was upsampled by using duplicatioon.

In [2]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale = 1./255, 
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('E:\\NUS\\Interview dataset\\orig\\Train',
target_size = (100, 100),
batch_size = 32,
classes = ['0','1'],                                                 
class_mode = 'binary')
test_set = test_datagen.flow_from_directory('E:\\NUS\\Interview dataset\\orig\\Test',
target_size = (100, 100),
batch_size = 32,
class_mode = 'binary')


Found 135 images belonging to 2 classes.
Found 125 images belonging to 2 classes.


In [3]:
print(test_set.image_shape)
Y_test = test_set.classes

(100, 100, 3)


In [4]:
Y_train = training_set.classes
training_set.image_shape

(100, 100, 3)

## Baseline Model

Simple CNN model with 2 layer convolutions with a filter size 3 * 3. The model runs for 25 epochs.

In [15]:
#Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (100, 100, 3), activation = 'relu')) 
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 16928)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               2166912   
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total para

In [33]:
classifier.fit_generator(training_set,
steps_per_epoch = 10,
epochs = 25,
validation_data = test_set,
validation_steps = 10)



Epoch 1/25
10/10 [==============================] - 56s 6s/step - loss: 0.6362 - acc: 0.6382 - val_loss: 0.5551 - val_acc: 0.7643
Epoch 2/25
10/10 [==============================] - 46s 5s/step - loss: 0.6052 - acc: 0.6618 - val_loss: 0.5117 - val_acc: 0.7878
Epoch 3/25
10/10 [==============================] - 40s 4s/step - loss: 0.5569 - acc: 0.7084 - val_loss: 0.5499 - val_acc: 0.7293
Epoch 4/25
10/10 [==============================] - 47s 5s/step - loss: 0.5789 - acc: 0.6921 - val_loss: 0.5278 - val_acc: 0.7685
Epoch 5/25
10/10 [==============================] - 43s 4s/step - loss: 0.5342 - acc: 0.7356 - val_loss: 0.4755 - val_acc: 0.8408
Epoch 6/25
10/10 [==============================] - 52s 5s/step - loss: 0.5649 - acc: 0.7406 - val_loss: 0.4944 - val_acc: 0.8457
Epoch 7/25
10/10 [==============================] - 48s 5s/step - loss: 0.5165 - acc: 0.7279 - val_loss: 0.4681 - val_acc: 0.8376
Epoch 8/25
10/10 [==============================] - 51s 5s/step - loss: 0.5652 - acc: 0.70

In [5]:
y = training_set.classes


### Confusion Matrix
The model doesn't classify label 3 ie class 2 properly because the number of images is half of first class. Hence in the following model we have created a dataset with 2 times the existing images of label 3 (upsampling). 


In [35]:
#Confution Matrix and Classification Report
#https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = classifier.predict_generator(test_set)
y_pred = np.round(Y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['0', '1']
print(classification_report(test_set.classes, y_pred, target_names=target_names))




Confusion Matrix
[[55 25]
 [33 12]]
Classification Report
             precision    recall  f1-score   support

          0       0.62      0.69      0.65        80
          3       0.32      0.27      0.29        45

avg / total       0.52      0.54      0.52       125




## Fine tuning parameters

In [14]:
# Fitting the CNN to the upsampled images for label 3

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('E:\\NUS\\Interview dataset\\orig\\Train',
target_size = (64, 64),
batch_size = 32,
class_mode = 'binary')
test_set = test_datagen.flow_from_directory('E:\\NUS\\Interview dataset\\orig\\Test',
target_size = (64, 64),
batch_size = 32,
class_mode = 'binary')
classifier.fit_generator(training_set,
steps_per_epoch = 50,
epochs = 25,
validation_data = test_set,
validation_steps = 50)


Found 500 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/25
50/50 [==============================] - 167s 3s/step - loss: 0.1693 - acc: 0.9266 - val_loss: 0.4673 - val_acc: 0.8418
Epoch 2/25
50/50 [==============================] - 149s 3s/step - loss: 0.1296 - acc: 0.9539 - val_loss: 0.5756 - val_acc: 0.8329
Epoch 3/25
50/50 [==============================] - 20647s 413s/step - loss: 0.1647 - acc: 0.9353 - val_loss: 0.4643 - val_acc: 0.8424
Epoch 4/25
50/50 [==============================] - 151s 3s/step - loss: 0.1367 - acc: 0.9384 - val_loss: 0.5370 - val_acc: 0.8242
Epoch 5/25
50/50 [==============================] - 150s 3s/step - loss: 0.1281 - acc: 0.9526 - val_loss: 0.4474 - val_acc: 0.8411
Epoch 6/25
50/50 [==============================] - 152s 3s/step - loss: 0.1107 - acc: 0.9620 - val_loss: 0.6536 - val_acc: 0.8001
Epoch 7/25
50/50 [==============================] - 152s 3s/step - loss: 0.1562 - acc: 0.9364 - val_loss: 0.5376 - val_acc: 0.80

In [15]:
#Confution Matrix and Classification Report
#https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = classifier.predict_generator(test_set, 120/ 33)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['0', '1']
print(classification_report(test_set.classes, y_pred, target_names=target_names))




Confusion Matrix
[[60  0]
 [60  0]]
Classification Report
             precision    recall  f1-score   support

          0       0.50      1.00      0.67        60
          3       0.00      0.00      0.00        60

avg / total       0.25      0.50      0.33       120



C:\Users\sownds\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
test_set.classes


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## 2. Deeper CNN 

A deeper model compared to previous simple CNN is used.

In [37]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])




In [41]:
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
plot_model(model, to_file='model_shapes.png', show_shapes=True)

In [38]:
model.fit_generator(training_set,
steps_per_epoch = 10,
epochs = 25,
validation_data = test_set,
validation_steps = 10)


Epoch 1/25
10/10 [==============================] - 58s 6s/step - loss: 1.6413 - acc: 0.5131 - val_loss: 0.7031 - val_acc: 0.3503
Epoch 2/25
10/10 [==============================] - 43s 4s/step - loss: 0.6971 - acc: 0.5015 - val_loss: 0.6467 - val_acc: 0.6945
Epoch 3/25
10/10 [==============================] - 43s 4s/step - loss: 0.7526 - acc: 0.4984 - val_loss: 0.6980 - val_acc: 0.3535
Epoch 4/25
10/10 [==============================] - 51s 5s/step - loss: 0.7152 - acc: 0.5105 - val_loss: 0.6353 - val_acc: 0.5723
Epoch 5/25
10/10 [==============================] - 42s 4s/step - loss: 0.7495 - acc: 0.5351 - val_loss: 0.6597 - val_acc: 0.6975
Epoch 6/25
10/10 [==============================] - 48s 5s/step - loss: 0.6941 - acc: 0.5215 - val_loss: 0.6502 - val_acc: 0.6688
Epoch 7/25
10/10 [==============================] - 47s 5s/step - loss: 0.7157 - acc: 0.5676 - val_loss: 0.6687 - val_acc: 0.7038
Epoch 8/25
10/10 [==============================] - 51s 5s/step - loss: 0.7681 - acc: 0.54

## Confusion Matrix

Similar results to Model 1 were obtained.

In [40]:
#Confution Matrix and Classification Report
#https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict_generator(test_set)
y_pred = np.round(Y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['0', '3']
print(classification_report(test_set.classes, y_pred, target_names=target_names))




Confusion Matrix
[[57 23]
 [33 12]]
Classification Report
             precision    recall  f1-score   support

          0       0.63      0.71      0.67        80
          3       0.34      0.27      0.30        45

avg / total       0.53      0.55      0.54       125

